# AEM Basic

This is a Python implementation of the Applied Element Method

In [17]:
# Basic AEM
# Note  all comments marked as #& indicate changes to be made in AEM_2D.py
#       all comments marked as #! are concerns to be addressed
#       all comments marked as ## indicates start and end of sections
#       all comments marked as #@ working comments

import numpy as np
from math import cos, sin


# Table of Contents
* [AEM Basic](#AEM-Basic)
	* [Input deck](#Input-deck)
	* [Stiffness Matrix Calculations](#Stiffness-Matrix-Calculations)
		* [Individual spring contributing calculations](#Individual-spring-contributing-calculations)
		* [Computing each element stiffness and global stiffness](#Computing-each-element-stiffness-and-global-stiffness)
	* [Calculating unknown forces and displacements](#Calculating-unknown-forces-and-displacements)
	* [Calculation of stresses and strains](#Calculation-of-stresses-and-strains)
		* [Calculating new co-ordinates of each spring](#Calculating-new-co-ordinates-of-each-spring)
		* [Adding element strains](#Adding-element-strains)
		* [Computing strains per element pair interaction](#Computing-strains-per-element-pair-interaction)
	* [Specifying Output](#Specifying-Output)


## Input deck

The input deck will in later versions be changed so that the recorded information is read from an input file written by a mesher.
Currently the units used for all geometry is in meters with all other material properties in SI units. Note that T indicates thickness or depth of material and nu is the Poisson ratio. The matrix el_center contains the element number and the x and y co-ordinates of the elements center.  Elements are numbered from the left to the right of the beam starting at element 0 and ending at element 9, thus total of 10 elements.  Should additional elements be added through the thickness of the beam the numbering continues again from left to right starting with element 10.

In [18]:
## Start of input deck
# Material properties
E = 207e+9
nu = 0.3
G = E/(2*(1+nu))
T = 0.15

# Element properties
#& Should be adjusted to load from hex_mesher
grid = [10, 1, 0]
num_ele = grid[0]*grid[1]
el_center = np.zeros(shape=(num_ele, 3))
el_count = 0
for y_c in range(100, 200, 100):
    for x_c in range(100, 2000, 200):
        el_center[el_count, 0] = el_count
        el_center[el_count, 1] = float(x_c)/1000
        el_center[el_count, 2] = float(y_c)/1000
        el_count += 1


a1 = 0.2         # width of element 1
b1 = 0.200         # height of element 1
theta1 = 0.      # radians (for time being not applicable)

#@ Seems redundent unless different size and shape elements
#& Remove in future releases
a2 = 0.2         # width of element 2
b2 = 0.200         # height of element 2
theta2 = 0.      # radians (for time being not applicable)

gap = [0.00, 0.00, 0]         # size of gap between elements

# Spring properties
#@ Assuming same number of connecting springs for horizontal and vetical
num_spring = 8

# Prescribed displacements
    # element number, DOF, value
Up = np.array([0, 0, 0])
#@ Needs to be done to read from input
dof = np.ones(num_ele*3)
dof[0] = 0
dof[1] = 0
dof[2] = 0

# Prescribed forces
    # element number, DOF, value
F = np.zeros(shape=(num_ele*3, 1))
F[28, 0] = -1.0e+3

## End of input deck

## Stiffness Matrix Calculations

The stiffness matrix is computed by adding the influence of each spring connecting adjacent elements.  The influence of a spring is computed as the forces it exert on the center of element for a unit displacement in the degree of freedom.  
Note the spring stiffnesses are what makes up the core of the method as each spring has to represent the material properties and the behavior of the material over the area wich is governed by d (distance between springs), a (distance from element center to element center) and T (thickness of the material). $$ K_n = \frac{EdT}{a} $$ 
$$ K_s = \frac{GdT}{a} $$

### Individual spring contributing calculations

Two separate functions were written to compute the individual affects of each spring on the element pair interaction stiffness.  These two functions are divided according to horizontal (Kte_row) interaction and vertical (Kte_column) interaction.

In [19]:
def Kte_row(Knh, Ksh, a1, a2, bn1, bn2):
    Kte = np.array([[Knh, 0, -Knh*bn1, -Knh, 0, Knh*bn2],
                   [0, Ksh, Ksh*(a1/2), 0, -Ksh, Ksh*(a2/2)],
                   [-Knh*bn1, Ksh*(a1/2),
                    Knh*bn1**2+Ksh*(a1/2)**2, Knh*bn1,
                    -Ksh*(a1/2), -Knh*bn1*bn2+Ksh*(a1/2)*(a2/2)],
                   [-Knh, 0, Knh*bn1, Knh, 0, -Knh*bn2],
                   [0, -Ksh, -Ksh*(a1/2), 0, Ksh, -Ksh*(a2/2)],
                   [Knh*bn2, Ksh*(a2/2),
                    (Knh*bn1*bn2)+Ksh*(a1/2)*(a2/2),
                    -Knh*bn2, -Ksh*(a2/2),
                    Knh*bn2**2+Ksh*(a2/2)**2]])
    return Kte

In [20]:
def Kte_column(Knv, Ksv, an1, an2, b1, b2):
    Kte = np.array([[Ksv, 0, -Ksv*(b1/2), -Ksv, 0, -Ksv*(b1/2)],
                    [0, Knv, -Knv*an1, 0, -Knv, Knv*an2],
                    [-Ksv*(b1/2), -Knv*an1,
                     Knv*an1**2+Ksv*(b1/2)**2, Ksv*b1,
                     Knv*an1, -Knv*an1*an2+Ksv*(b1/2)*(b2/2)],
                    [-Ksv, 0, Ksv*(b1/2), Ksv, 0, Ksv*(b2/2)],
                    [0, -Knv, Knv*an1, 0, Knv, -Knv*an2],
                    [-Ksv*(b2/2), Knv*an2,
                     -Ksv*an1*an2+Ksv*(b1/2)*(b2/2),
                     Ksv*(b2/2), -Knv*an2,
                     Knv*an2**2+Ksv*(b2/2)**2]])
    return Kte

### Computing each element stiffness and global stiffness

The above shown functions are used to compute the element pair stifness and then the global stiffness.  Note that the use of the translation matrix and the rotation of elements to and from the global co-ordinates are still a bit unsure.  The influence of different size elements also remain a question to be answered.  The influence of the element pair interaction is assigned to the global stiffness matrix in accordance with the degrees of freedom represented where degrees 0-2 are element zero, 3-5 element one and so forth.  Elements numbers are assigned along the positive x axis in cronological order.

In [21]:
## Start of calculating stiffness matricies [Prashidha 2014]

#@ Insert incrementation here
#@ Assume all elements of same size currently
#@ For horizontal element interaction
dh = float(b1)/(num_spring)
#! Not sure how gap plays role here
ah = a1/2 + a2/2
Knh = (E*dh*T)/ah
Ksh = (G*dh*T)/ah

#@ For vertical element interaction
dv = float(a1)/(num_spring)
#! Not sure how gap plays role here
av = b1/2 + b2/2
Knv = (E*dv*T)/av
Ksv = (G*dv*T)/av

#! Unsure of the workings of theta1 and theta2
#! Question 3: Setting up the stiffness matrices, these do not change unless a
                # spring is no longer active
#@ Note stiffness matrices are compiled per spring thus per element pair
Kg = np.zeros(shape=(num_ele*3, num_ele*3))
elemn = 0
# Translation matrix:
#! Check use of theta2
L = np.array([[cos(theta1), sin(theta1), 0, 0, 0, 0],
              [-sin(theta1), cos(theta1), 0, 0, 0, 0],
              [0, 0, 1, 0, 0, 0],
              [0, 0, 0, cos(theta2), sin(theta2), 0],
              [0, 0, 0, -sin(theta2), cos(theta2), 0],
              [0, 0, 0, 0, 0, 1]])
LT = L.T
## Start loops over element pairs
for row in range(0, grid[1]-1):
    for column in range(0, grid[0]-1):
        ## Start of horizontal element pair interaction
        Kel = np.zeros(shape=(6, 6))
        for sign in range(1, 3):
            for n in range(1, (num_spring/2)+1):
                bn1 = ((-1)**sign)*(dh/2 + dh*(n-1))
                bn2 = ((-1)**sign)*(dh/2 + dh*(n-1))
                Kte = Kte_row(Knh, Ksh, a1, a2, bn1, bn2)
                Kel = Kel + Kte     # adding each spring's contribution
        Ke = LT.dot(Kel.dot(L))
        ## End of horizontal element pair interaction
        ## Adding element pair stiffness to global stiffness
        pg_r = ([3*elemn, 3*elemn+1, 3*elemn+2,
                 3*(elemn+1), 3*(elemn+1)+1, 3*(elemn+1)+2])
        Kg[np.ix_(pg_r, pg_r)] = Kg[np.ix_(pg_r, pg_r)] + Ke

        ## Start of vertical element pair interaction
        Kel = np.zeros(shape=(6, 6))
        for sign in range(1, 3):
            for n in range(1, (num_spring/2)+1):
                an1 = ((-1)**sign)*(dv/2 + dv*(n-1))
                an2 = ((-1)**sign)*(dv/2 + dv*(n-1))
                Kte = Kte_column(Knv, Ksv, an1, an2, b1, b2)
                Kel = Kel + Kte     # adding each spring's contribution

        Ke = LT.dot(Kel.dot(L))
        ## End of vertical element pair interaction
        ## Adding element pair stiffness to global stiffness
        pg_c = ([3*elemn, 3*elemn+1, 3*elemn+2,
                 3*(elemn+grid[0]), 3*(elemn+grid[0])+1, 3*(elemn+grid[0])+2])
        Kg[np.ix_(pg_c, pg_c)] = Kg[np.ix_(pg_c, pg_c)] + Ke
        elemn += 1

    ## Start of vertical element pair interaction (interaction for last(left most) column)
    Kel = np.zeros(shape=(6, 6))
    for sign in range(1, 3):
        for n in range(1, (num_spring/2)+1):
            an1 = ((-1)**sign)*(dv/2 + dv*(n-1))
            an2 = ((-1)**sign)*(dv/2 + dv*(n-1))
            Kte = Kte_column(Knv, Ksv, an1, an2, b1, b2)
            Kel = Kel + Kte     # adding each spring's contribution

    Ke = LT.dot(Kel.dot(L))
    ## End of vertical element pair interaction
    ## Adding element pair stiffness to global stiffness
    pg_c = ([3*elemn, 3*elemn+1, 3*elemn+2,
             3*(elemn+grid[0]), 3*(elemn+grid[0])+1, 3*(elemn+grid[0])+2])
    Kg[np.ix_(pg_c, pg_c)] = Kg[np.ix_(pg_c, pg_c)] + Ke
    elemn += 1
## Start of horizontal element pair interaction (interaction for last(top) row)
for column in range(0, grid[0]-1):
    Kel = np.zeros(shape=(6, 6))
    for sign in range(1, 3):
        for n in range(1, (num_spring/2)+1):
            bn1 = ((-1)**sign)*(dh/2 + dh*(n-1))
            bn2 = ((-1)**sign)*(dh/2 + dh*(n-1))
            Kte = Kte_row(Knh, Ksh, a1, a2, bn1, bn2)
            Kel = Kel + Kte     # adding each spring's contribution
    Ke = LT.dot(Kel.dot(L))
    ## End of horizontal element pair interaction
    ## Adding element pair stiffness to global stiffness
    pg_r = ([3*elemn, 3*elemn+1, 3*elemn+2,
             3*(elemn+1), 3*(elemn+1)+1, 3*(elemn+1)+2])
    Kg[np.ix_(pg_r, pg_r)] = Kg[np.ix_(pg_r, pg_r)] + Ke
    elemn += 1
## End of calculating stiffness matrices

## Calculating unknown forces and displacements

To calculate the unknowns in the equation $$ K.U=F $$ the stiffness matrix and coresponding degrees of freedom and forces needs to be split into prescribed displacements and free displacements.  Since we know that F for the free to displace degrees of freedom is 0 and we know the prescribed displacements the effect of the prescribed displacements on the forces for these degrees of freedom can be calculated  $$ F_{p1} = F_f - K_{fp}.U_p $$  This can then be used to calcuate the free displacements with $$ K_{ff}.U_f = F_{p1} $$ and finally the forces for the prescribed degrees of freedom can be calculated using $$ F_p = K_{fp}^{T}.U_f + K_{pp}.U_p $$

In [22]:
## Start of calculating unknown degrees of freedom and reaction forces

# Next the stiffness matrix will be devided so that the free and
    # prescribed degree of freedoms can be idependantly used to calculate
    # the unknown variables

# Loads are applied at the centre of an element on the degrees of freedom

# Solve displacements
U = np.zeros(shape=(num_ele*3, 1))
# To solve for the free displacements the stiffness matrix needs to be devided
    # this is done by clasifying the free and prescribed degrees of freedom
pdof = np.where(dof == 0)[0]
fdof = np.nonzero(dof)[0]

Kff = Kg[fdof][:, fdof]
Kfp = Kg[fdof][:, pdof]
Kpp = Kg[pdof][:, pdof]

# Now we include the effects of prescribed displacements
Fp1 = F[fdof, 0] - Kfp.dot(Up)
# Solve unknown degrees of freedom
Uf = np.linalg.solve(Kff, Fp1)
# Finally solve for the support reactions
KfpT = Kfp.T
Fp = KfpT.dot(Uf) + Kpp.dot(Up)
# Placing calculated values back into global F and U
U[fdof, 0] = Uf
U[pdof, 0] = Up
F[pdof, 0] = Fp
## End of calculating unknown degrees of freedom and reaction forces

## Calculation of stresses and strains

The calculation of the stresses and strains are once again done per element pair interaction as for the stiffness matrix.  The strains are calculated using the displacement of each spring divided by the distance between the element pair centers.  Strains are calculated using the infinitesimal strain tensor, for small strains. Thus $$ \epsilon_{11} = \frac{u_1}{x_1} $$ $$ \epsilon_{22} = \frac{u_2}{x_2} $$ $$ \epsilon_{12} = \frac{u_1}{x_2} + \frac{u_2}{x_1} $$ The strains are added together using a function for the horizontal and a function for the vertical interactions.  Also the new co-ordinates of each spring is calculated in a separate function.

The stress are calculated according to the plane stress and plane strain principals however since the thickness of the material is taken into account for each spring's stiffness I am unsure if this is correct.

### Calculating new co-ordinates of each spring

In [23]:
def new_coordinates(num_spring, ele, x_co, y_co, U, dof1, dof2):
    Li = np.zeros(shape=(num_spring, 2))
    x_co_n = np.zeros(shape=(num_spring, 2))  # [spring num,element num]
    y_co_n = np.zeros(shape=(num_spring, 2))  # [spring num,element num]
    x_co_n[:, 0] = x_co[:, 0] + U[dof1, 0]
    x_co_n[:, 1] = x_co[:, 1] + U[dof2, 0]
    y_co_n[:, 0] = y_co[:, 0] + U[dof1+1, 0]
    y_co_n[:, 1] = y_co[:, 1] + U[dof2+1, 0]
    for spr in range(0, num_spring):
        x_co_n[spr, 0] = (x_co_n[spr, 0]*cos(U[dof1+2, 0])
                          - y_co_n[spr, 0]*sin(U[dof1+2, 0]))
        x_co_n[spr, 1] = (x_co_n[spr, 1]*cos(U[dof2+2, 0])
                          - y_co_n[spr, 1]*sin(U[dof2+2, 0]))
        y_co_n[spr, 0] = (x_co_n[spr, 0]*sin(U[dof1+2, 0])
                          + y_co_n[spr, 0]*cos(U[dof1+2, 0]))
        y_co_n[spr, 1] = (x_co_n[spr, 1]*sin(U[dof2+2, 0])
                          + y_co_n[spr, 1]*cos(U[dof2+2, 0]))
        Li[spr, 0] = x_co_n[spr, 1] - x_co_n[spr, 0]    # New length of x
        Li[spr, 1] = y_co_n[spr, 1] - y_co_n[spr, 0]    # New length of y
    return Li, x_co_n, y_co_n

### Adding element strains

In [24]:
def element_strain_column(strain_ele, strain_column, ele, grid):
    strain_ele[0, ele] = (strain_ele[0, ele]
                          + np.average(strain_column[0, :]))
    strain_ele[1, ele] = (strain_ele[1, ele]
                          + np.average(strain_column[1, :]))
    strain_ele[2, ele] = (strain_ele[2, ele]
                          + np.average(strain_column[2, :]))

    strain_ele[0, ele+grid[0]] = (strain_ele[0, ele+grid[0]]
                                  + np.average(strain_column[0, :]))
    strain_ele[1, ele+grid[0]] = (strain_ele[1, ele+grid[0]]
                                  + np.average(strain_column[1, :]))
    strain_ele[2, ele+grid[0]] = (strain_ele[2, ele+grid[0]]
                                  + np.average(strain_column[2, :]))
    return strain_ele

In [25]:
def element_strain_row(strain_ele, strain_row, ele, grid):
    strain_ele[0, ele] = (strain_ele[0, ele]
                          + np.average(strain_row[0, :]))
    strain_ele[1, ele] = (strain_ele[1, ele]
                          + np.average(strain_row[1, :]))
    strain_ele[2, ele] = (strain_ele[2, ele]
                          + np.average(strain_row[2, :]))

    strain_ele[0, ele+1] = (strain_ele[0, ele+1]
                            + np.average(strain_row[0, :]))
    strain_ele[1, ele+1] = (strain_ele[1, ele+1]
                            + np.average(strain_row[1, :]))
    strain_ele[2, ele+1] = (strain_ele[2, ele+1]
                            + np.average(strain_row[2, :]))
    return strain_ele

### Computing strains per element pair interaction

The original co-ordinates of each spring is calculated and then used along with the displacements to calculate the new co-ordinates.  Using this the strains are calculated and finally the stresses are calculated using both the plane strain and plane stress approximation.

In [26]:
## Start of calculating stresses and strains
ele = 0
strain_ele = np.zeros(shape=(3, grid[0]*grid[1]))   # [strain; ele]

L0_row = np.zeros(shape=(num_spring, 2))
dL_row = np.zeros(shape=(num_spring, 2))
strain_row = np.zeros(shape=(3, num_spring))

L0_column = np.zeros(shape=(num_spring, 2))
dL_column = np.zeros(shape=(num_spring, 2))
strain_column = np.zeros(shape=(3, num_spring))
## Loop over elements
for row in range(0, grid[1]-1):
    for column in range(0, grid[0]-1):
        # Calculating the original co-ordinates of each spring for horizontal
        x_co = np.zeros(shape=(num_spring, 2))  # [spring num, element num]
        y_co = np.zeros(shape=(num_spring, 2))  # [spring num, element num]
        spr = 0
        for sign in range(1, 3):
            for n in range(1, (num_spring/2)+1):
                bn1 = ((-1)**sign)*(dh/2 + dh*(n-1))
                bn2 = ((-1)**sign)*(dh/2 + dh*(n-1))
                x_co[spr, 0] = el_center[ele, 1] + a1/2
                x_co[spr, 1] = el_center[ele+1, 1] - a2/2
                y_co[spr, 0] = el_center[ele, 2] + bn1
                y_co[spr, 1] = el_center[ele+1, 2] + bn2
                L0_row[spr, 0] = x_co[spr, 1] - x_co[spr, 0]    # Original L N
                L0_row[spr, 1] = y_co[spr, 1] - y_co[spr, 0]    # Original L S
                spr += 1
        dof1 = ele*3
        dof2 = (ele+1)*3
        Li_row, x_co_n, y_co_n = new_coordinates(num_spring, ele, x_co, y_co, U, dof1, dof2)
        dL_row[:, 0] = Li_row[:, 0] - L0_row[:, 0]  # x direction
        dL_row[:, 1] = Li_row[:, 1] - L0_row[:, 1]  # y direction
        strain_row[0, :] = dL_row[:, 0]/(L0_row[:, 0] + ah)
        strain_row[2, :] = dL_row[:, 1]/(L0_row[:, 0] + ah)

        # Calculating the original co-ordinates of each spring for vertical
        x_co = np.zeros(shape=(num_spring, 2))  # [spring num, element num]
        y_co = np.zeros(shape=(num_spring, 2))  # [spring num, element num]
        spr = 0
        for sign in range(1, 3):
            for n in range(1, (num_spring/2)+1):
                an1 = ((-1)**sign)*(dv/2 + dv*(n-1))
                an2 = ((-1)**sign)*(dv/2 + dv*(n-1))
                x_co[spr, 0] = el_center[ele, 1] + an1
                x_co[spr, 1] = el_center[ele+grid[0], 1] + an2
                y_co[spr, 0] = el_center[ele, 2] + b1/2
                y_co[spr, 1] = el_center[ele+grid[0], 2] - b2/2
                L0_column[spr, 0] = x_co[spr, 1] - x_co[spr, 0]  # Original L S
                L0_column[spr, 1] = y_co[spr, 1] - y_co[spr, 0]  # Original L N
                spr += 1
        dof1 = ele*3
        dof2 = (ele+grid[0])*3
        Li_column, x_co_n, y_co_n = new_coordinates(num_spring, ele, x_co, y_co, U, dof1, dof2)
        dL_column[:, 0] = Li_column[:, 0] - L0_column[:, 0]     # x direction
        dL_column[:, 1] = Li_column[:, 1] - L0_column[:, 1]     # y direction
        strain_column[1, :] = dL_column[:, 1]/(L0_column[:, 0] + av)
        strain_column[2, :] = dL_column[:, 0]/(L0_column[:, 0] + av)

        strain_ele = element_strain_column(strain_ele,
                                           strain_column, ele, grid)
        strain_ele = element_strain_row(strain_ele, strain_row, ele, grid)
        ele += 1
    # Calculating the original co-ordinates of each spring for vertical
    x_co = np.zeros(shape=(num_spring, 2))  # [spring num, element num]
    y_co = np.zeros(shape=(num_spring, 2))  # [spring num, element num]
    spr = 0
    for sign in range(1, 3):
        for n in range(1, (num_spring/2)+1):
            an1 = ((-1)**sign)*(dv/2 + dv*(n-1))
            an2 = ((-1)**sign)*(dv/2 + dv*(n-1))
            x_co[spr, 0] = el_center[ele, 1] + an1
            x_co[spr, 1] = el_center[ele+grid[0], 1] + an2
            y_co[spr, 0] = el_center[ele, 2] + b1/2
            y_co[spr, 1] = el_center[ele+grid[0], 2] - b2/2
            L0_column[spr, 0] = x_co[spr, 1] - x_co[spr, 0]  # Original L S
            L0_column[spr, 1] = y_co[spr, 1] - y_co[spr, 0]  # Original L N
            spr += 1
    dof1 = ele*3
    dof2 = (ele+grid[0])*3
    Li_column, x_co_n, y_co_n = new_coordinates(num_spring, ele, x_co, y_co, U, dof1, dof2)
    dL_column[:, 0] = Li_column[:, 0] - L0_column[:, 0]
    dL_column[:, 1] = Li_column[:, 1] - L0_column[:, 1]
    strain_column[1, :] = dL_column[:, 1]/(L0_column[:, 0] + av)
    strain_column[2, :] = dL_column[:, 0]/(L0_column[:, 0] + av)

    strain_ele = element_strain_column(strain_ele,
                                       strain_column, ele, grid)

    ele += 1
for column in range(0, grid[0]-1):
    # Calculating the original co-ordinates of each spring for horizontal
    x_co = np.zeros(shape=(num_spring, 2))  # [spring num, element num]
    y_co = np.zeros(shape=(num_spring, 2))  # [spring num, element num]
    spr = 0
    for sign in range(1, 3):
        for n in range(1, (num_spring/2)+1):
            bn1 = ((-1)**sign)*(dh/2 + dh*(n-1))
            bn2 = ((-1)**sign)*(dh/2 + dh*(n-1))
            x_co[spr, 0] = el_center[ele, 1] + a1/2
            x_co[spr, 1] = el_center[ele+1, 1] - a2/2
            y_co[spr, 0] = el_center[ele, 2] + bn1
            y_co[spr, 1] = el_center[ele+1, 2] + bn2
            L0_row[spr, 0] = x_co[spr, 1] - x_co[spr, 0]    # Original L N
            L0_row[spr, 1] = y_co[spr, 1] - y_co[spr, 0]    # Original L S
            spr += 1
    dof1 = ele*3
    dof2 = (ele+1)*3
    Li_row, x_co_n, y_co_n = new_coordinates(num_spring, ele, x_co, y_co, U, dof1, dof2)
    dL_row[:, 0] = Li_row[:, 0] - L0_row[:, 0]  # x direction
    dL_row[:, 1] = Li_row[:, 1] - L0_row[:, 1]  # y direction
    strain_row[0, :] = dL_row[:, 0]/(L0_row[:, 0] + ah)
    strain_row[2, :] = dL_row[:, 1]/(L0_row[:, 0] + ah)

    strain_ele = element_strain_row(strain_ele, strain_row, ele, grid)
    ele += 1

#@ Stress calculation per element
stress_Pstrain = np.zeros(shape=(3, grid[0]*grid[1]))
stress_Pstress = np.zeros(shape=(3, grid[0]*grid[1]))
C_strain = np.array([[1-nu, nu, 0],
                     [nu, 1-nu, 0],
                     [0, 0, 1-2*nu]])
C_stress = np.array([[1, nu, 0],
                     [nu, 1, 0],
                     [0, 0, 1-nu]])

stress_Pstrain = (E/((1+nu)*(1-2*nu))) * C_strain.dot(strain_ele)
stress_Pstress = (E/(1-nu**2)) * C_stress.dot(strain_ele)
## End of calculating stresses and strains

## Specifying Output

Later editions will include visual representation for easier verification and validation

In [27]:
print('U=')
print(U)
print('F=')
print(F)
print('strain=')
print(strain_ele)
print('Pstrain=')
print(stress_Pstrain)
print('Pstress=')
print(stress_Pstress)

U=
[[  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [ -2.66325146e-24]
 [ -2.22473349e-07]
 [ -1.38737439e-06]
 [ -1.55818470e-24]
 [ -5.26117613e-07]
 [ -8.11709142e-07]
 [ -2.01130265e-24]
 [ -7.95799740e-07]
 [ -1.04775303e-06]
 [ -1.81247179e-24]
 [ -1.07872851e-06]
 [ -9.44175565e-07]
 [ -1.86792802e-24]
 [ -1.35418843e-06]
 [ -9.73064519e-07]
 [ -1.78000961e-24]
 [ -1.62795729e-06]
 [ -9.27264958e-07]
 [ -1.65962902e-24]
 [ -1.89087517e-06]
 [ -8.64554789e-07]
 [ -1.33094944e-24]
 [ -2.13040005e-06]
 [ -6.93334892e-07]
 [ -5.53209674e-25]
 [ -2.31228794e-06]
 [ -2.88184930e-07]]
F=
[[    0.        ]
 [ 1000.        ]
 [  -41.34960517]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.       